In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [92]:
def face_detection(image):
    #cv2.imshow('2',image)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_frontalface_default.xml')

    face = haar_classifier.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=7)
    #eye = 
    #print(face)
    (x,y,w,h) = face[0]
    return image_gray[y:y+w, x:x+h], face[0]

#eye_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_eye.xml') 
#eye = eye_cascade.detectMultiScale(img2)
#(x,y,w,h) = eye[0]

In [93]:
# Apply LBP Algorithm on a block of the image
def get_lbp_hist(grayscale_img):
    
    dim = grayscale_img.shape
    
    lbp = np.zeros(256)
    
    num1=num2=0
    
    
    #print(grayscale_img.shape)
    for i in range(1,dim[0]-1):
        for j in range(1,dim[1]-1):
            
            num1+=1
            binary = ""
            if grayscale_img[i][j] < grayscale_img[i-1][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_imgyscale_img[i][j] < grayscale_img[i-1][j]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i-1][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            
            transitions = 0
            
            for k in range(1,len(binary)):
                if binary[k] != binary[k-1]:
                    transitions +=1
            
            if transitions  > 2: 
                continue
                
            num2+=1
            
            binary = int(binary,2)
            lbp[binary]+=1
    
    return lbp # <LBP histogram which is a list of 256 numbers>

In [94]:
# Segment the image into 7x7 blocks,apply LBP algorithm on them, then Concatente all hitograms into one
def segment_img(img):
    patch_width  = 7
    patch_height = 7
    dim = img.shape
    
    histograms = []
    num = 0
    
    for x in range(0,dim[0],patch_width):
        for y in range(0,dim[1],patch_height):
            
            
            if(patch_width+x >= dim[0] or patch_width+y >= dim[1] ):
                continue
            num+=1 
            patch = img[x:x+patch_width,y:y+patch_height]
    
            histogram = get_lbp_hist(patch)
            histograms.append(histogram)
    
    histo = np.zeros(256)
    
    for h in histograms:
        for i in range(len(h)):
            histo[i] += h[i]
        
        
    return histo

In [100]:
# Training on images for "tom" and "clinton"

classes = []
train_hist  = []
train_labels = []

def train_data():
    
    
    for person in os.listdir('training'):
        
        classes.append(person)
        
        for image in os.listdir('training/'+person):
            
            img_path = 'training/'+person+'/'+image
            img = cv2.imread(img_path)
            img,_ = face_detection(img)
        
            histo = segment_img(img)
            train_hist.append(histo)
            train_labels.append(person)

        
'''    
    for image in os.listdir('tom'):
        
        img_path = 'tom/' + image
        img = cv2.imread(img_path)
        img,_ = face_detection(img)
        
        histo = segment_img(img)
        
        train_hist.append(histo)
        train_labels.append(1)

    for image in os.listdir('clinton'):
        
        img_path = 'clinton/' + image
        img = cv2.imread(img_path)
        img,_ = face_detection(img)
        
        histo = segment_img(img)
        
        train_hist.append(histo)
        train_labels.append(2)
        #print(len(train_labels))
    
        #cv2.imshow('xxx',img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
'''
train_data()

In [103]:
#print(train_labels)

In [96]:
def KNN(all_hist,test_hist,k):
    pass


# Apply Nearest Neighbour Algorithm for test image 
def test_img(img):
    
    img,_ = face_detection(img)
    
    test_hist = segment_img(img)
    
    
    mini_dist  = 1000000
    mini_class = -1
    
    for index in range(len(train_hist)) :
        d = 0 
        
        for i in range(256):
            if train_hist[index][i] != 0:
                d += ( ( train_hist[index][i] - test_hist[i] )**2 ) // train_hist[index][i]
             
        if d < mini_dist :
            mini_dist = d 
            mini_class = train_labels[index]
            
            
    print(mini_dist)
    
    if mini_dist > 1000:
        return -1
    return mini_class

In [97]:
train_data()

In [106]:
img = cv2.imread('test/9.jpg')
#cv2.imshow('Test Image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

c = test_img(img)    
if c != -1:
    print (c)
else:
    print("No Match")

422.0
tom
